In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import log_loss, make_scorer, confusion_matrix, mean_squared_error
from pandas.plotting import scatter_matrix
from regression_tools.plotting_tools import plot_univariate_smooth




In [2]:
amazon_df = pd.read_csv('../data/clean_amazon_reviews.csv')

,Unnamed: 0,Unnamed: 0.1,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,...,career-opportunities-stars,comp-benefit-stars,senior-management-stars,helpful-count,role,is_current_employee,year,month,quarter,amazon_earnings_this_quarter
0,7819,7821,amazon,"Sunnyvale, CA","Dec 10, 2018","Current Employee - Manager, Software Developme...",Software Development Engineering Manager,"Work hard, have fun, make history. Be proud of...",RUS grand each year related to last year's sto...,none,...,5.000000,4.00000,5.000000,0,"Manager, Software Development &amp, Engineering",1,2018,Dec,4,3.027
1,7820,7822,amazon,"Seattle, WA","Jan 10, 2016",Current Employee - Anonymous Employee,You Get What You Put In,"Really smart people, a lot of opportunity for ...",You have to be self motivated. NO ONE will hol...,More on-boarding training before new employees...,...,5.000000,5.00000,5.000000,1759,Anonymous Employee,1,2016,Jan,1,0.513
2,7821,7823,amazon,"Seattle, WA","Feb 20, 2016",Current Employee - Senior Engineering Manager,"Exciting Work, Abusive Culture","Jeff Bezos and his ""S-Team"" are brilliant and ...","The management process is abusive, and I'm cur...",Don't pretend that the recent NY Times article...,...,3.000000,4.00000,4.000000,1404,Senior Engineering Manager,1,2016,Feb,1,0.513
3,7822,7824,amazon,"Seattle, WA","Dec 17, 2013",Current Employee - Software Development Manager,"Can be amazing for some people, horrible for o...",Amazon is doing lot's of cool stuff...but lots...,- You're responsible for your own career progr...,Stack ranking is a horrible practice since it'...,...,5.000000,5.00000,4.000000,1275,Software Development Manager,1,2013,Dec,4,0.240
4,7823,7825,amazon,"Seattle, WA","Mar 23, 2017",Former Employee - Anonymous Employee,We are now in a world where we are condescende...,This company gets A list performance from C li...,Maybe because the company recruits C listers w...,Bezos you bought the Washington Post and hired...,...,1.000000,5.00000,1.000000,409,Anonymous Employee,0,2017,Mar,1,0.724
5,7824,7826,amazon,"Carlisle, PA","May 15, 2017",Former Employee - Tier I Warehouse Associate,"Warehouse associate, full time for over two years",Great starting pay and the ability to get rais...,After 3 years of working with the company you ...,none,...,4.000000,4.00000,5.000000,257,Tier I Warehouse Associate,0,2017,May,2,0.197
6,7825,7827,amazon,"Seattle, WA","Mar 29, 2017",Current Employee - Finance Manager,"Great place to learn, churn n burn culture in ...","- super smart people, the best of the best fro...","- frugality is taken to the extreme, only 2 we...",The culture and reputation of amazon will neve...,...,4.000000,3.00000,3.000000,245,Finance Manager,1,2017,Mar,1,0.724
7,7826,7828,amazon,"Seattle, WA","Dec 7, 2018",Current Employee - Senior Product Manager,Sr. Product manager,Smartest people I’ve ever worked with. The abi...,"Not a con, just hard work. But hard rewarding ...",Reconsider the base salary cap and lack of bas...,...,5.000000,5.00000,4.000000,2,Senior Product Manager,1,2018,Dec,4,3.027
8,7827,7829,amazon,none,"Dec 10, 2018",Current Employee - Anonymous Employee,Central operations,"Weekly pay, great pay,flexible shifts","Fighting for hours, unorganized and bad training",none,...,4.000000,5.00000,5.000000,1,Anonymous Employee,1,2018,Dec,4,3.027
9,7828,7830,amazon,none,"Dec 11, 2018",Current Employee - Warehouse Sortation Associate,Warehouse Sortation Associate,- Company works like a well oiled machine - Ma...,- A lot of the workers don't really care and h...,"Don't just hire everyone, put a bit more effor...",...,3.000000,4.00000,5.000000,0,Warehouse Sortation Associate,1,2018,Dec,4,3.027


In [3]:
# Not going to use dates, job-title, role, overall-ratings, location, month
# for now

X = amazon_df[['culture-values-stars', 'career-opportunities-stars', 
                       'comp-benefit-stars', 'senior-management-stars', 'helpful-count',
                       'is_current_employee', 'year', 'quarter', 'amazon_earnings_this_quarter']]
y = amazon_df['work-balance-stars'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=0)

### Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
sklearn.metrics.mean_squared_error

### Random Forest

In [24]:
random_forest_grid = {'max_depth': [3, None],
                      'max_features': ['sqrt', 'log2', None],
                      'min_samples_split': [2, 4],
                      'min_samples_leaf': [1, 2, 4],
                      'bootstrap': [True, False],
                      'n_estimators': [100, 200, 500, 1000],
                      'random_state': [1]}

rf_gridsearch = GridSearchCV(RandomForestRegressor(),
                             random_forest_grid,
                             n_jobs=-1,
                             verbose=True,
                             scoring='neg_mean_squared_error')
rf_gridsearch.fit(X_train, y_train)
print( "best parameters:", rf_gridsearch.best_params_ )

best_rf_model = rf_gridsearch.best_estimator_


Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 32.3min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 39.9min finished


best parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 1}


In [27]:
best_params = {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 1}
rf = RandomForestRegressor(**best_params)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [28]:
y_pred = rf.predict(X_test)

In [33]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(rmse)

0.93692045248394


### Gradient Boosted Regressor

In [44]:
gbr = GradientBoostingRegressor(learning_rate=0.005,
                                   n_estimators=1000,
                                   min_samples_leaf=5,
                                   max_depth=3,
                                   subsample=0.5)
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.005, loss='ls', max_depth=3,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=5, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=None, subsample=0.5, verbose=0,
             warm_start=False)

In [45]:
y_pred = gbr.predict(X_test)

In [47]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(rmse)

0.9308797464710922


### Later

In [ ]:
_ = scatter_matrix(amazon_df, alpha=0.2, figsize=(10, 10), diagonal='kde')

In [ ]:


fig, ax = plt.subplots(figsize=(14, 3))

plot_univariate_smooth(
    ax, 
    non_zero_bal_df["limit"].values.reshape(-1, 1),
    non_zero_bal_df['balance'],
    bootstrap=100)

ax.set_title("Univariate Effect of Credit Limit on Non-zero Bank Balance")
ax.set_ylabel("Non-zero Balance")
ax.set_xlabel("Limit")

In [ ]:
limit_fit = Pipeline([
    ('limit', ColumnSelector(name='limit')),
    ('limit_spline', LinearSpline(knots=[2500, 6000, 7000]))
])